<a href="https://colab.research.google.com/github/nguyenvudev20/mse22.Coursera_ML_AI/blob/main/AssignmentEnd3.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Housing Prices Competition for Kaggle Learn Users

Housing Prices Competition for Kaggle Learn Users là một bộ dữ liệu được cung cấp bởi Kaggle, một nền tảng nổi tiếng về các cuộc thi về khoa học dữ liệu và máy học. Bộ dữ liệu này bao gồm thông tin về giá nhà ở Ames, Iowa, Hoa Kỳ.

Bộ dữ liệu này bao gồm hai tập dữ liệu, một tập dữ liệu huấn luyện và một tập dữ liệu kiểm tra. Tập dữ liệu huấn luyện bao gồm 1460 mẫu và 80 biến đầu vào, bao gồm các thông tin về kích thước, vị trí, tiện nghi và trạng thái của ngôi nhà. Biến đầu ra là giá trị của ngôi nhà. Tập dữ liệu kiểm tra bao gồm 1459 mẫu và cũng bao gồm các biến đầu vào nhưng không có biến đầu ra.

Bộ dữ liệu này được sử dụng để đào tạo các mô hình dự đoán giá nhà dựa trên các đặc trưng của ngôi nhà. Nhiệm vụ của các thí sinh là tìm ra một mô hình tốt nhất để dự đoán giá trị của các ngôi nhà trong tập dữ liệu kiểm tra. Bộ dữ liệu này là một trong những bộ dữ liệu phổ biến và được sử dụng nhiều trong lĩnh vực dự đoán giá nhà và phân tích dữ liệu.


## Table of Contents
- [1 - Cài đặt dữ liệu từ Kaggle](#1)
- [2 - Lựa chọn và chuẩn hóa dữ liệu cần cho việc training](#2)
- [3 - Thực hiện trainning dựa trên model của Tensorflow Keras](#3)
- [4 - Sử dụng Hyperparameter Tuning để cải tiến model](#4)
- [5 - Cải tiến Hyperparameter Tuning](#5)
- [6 - Cải tiến bằng cách dùng Hyperband](#6)

<a name='1'></a>
## 1 - Cài đặt dữ liệu từ Kaggle

In [1]:
!pip install --upgrade scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import pandas as pd
import numpy as np
# tạo các tham số môi trường cho api kaggle để kết nối tải file 
os.environ['KAGGLE_USERNAME'] = "vunguyen2020"
os.environ['KAGGLE_KEY'] = "ceaa092a89951b4d01485d4ee50cf382"

In [3]:
!kaggle competitions download -c home-data-for-ml-course #tải gói data cho bài tutorial từ kaggle dưới dạng zip file
!mkdir kaggle #tạo folder để unzip file vừa tải
!unzip /content/home-data-for-ml-course.zip -d kaggle #tiến hành unzip vào folder kaggle

home-data-for-ml-course.zip: Skipping, found more recently modified local copy (use --force to force download)
mkdir: cannot create directory ‘kaggle’: File exists
Archive:  /content/home-data-for-ml-course.zip
replace kaggle/data_description.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

<a name='2'></a>
## 2 - Lựa chọn và chuẩn hóa dữ liệu cần cho việc training

In [ ]:
# Đọc dữ liệu từ file train.csv
train_data = pd.read_csv("kaggle/train.csv")
# Hiển thị 5 dòng đầu tiên của dữ liệu
train_data.head()

In [ ]:
# xem thucộ tính của các trường dữ liệu
train_data.info()

In [ ]:
# dữ liệu có 1460 dòng và 81 trường dữ liệu
train_data.shape

In [ ]:
# lựa chọn các trường dữ liệu
train_data.columns

In [ ]:
features = ['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
    '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr',
    'KitchenAbvGr', 'TotRmsAbvGrd', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
    '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
X= train_data[features]
y=train_data['SalePrice']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
# kiểm tra dữ liệu rỗng
X.isnull().sum().sort_values(ascending = False)

<a name='3'></a>
## 3 - Thực hiện trainning dựa trên model của Tensorflow Keras

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra với tỷ lệ 8:2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu đầu vào
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError
import tensorflow.keras.backend as K

# tạo hàm r_squared để đánh giá model
def r_squared(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# Khỏi tạo model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[MeanSquaredError(), RootMeanSquaredError(), MeanAbsoluteError(), r_squared])

# Huấn luyện model trên tập huấn luyện
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))



In [ ]:
# chạy thử mô hình với tập kiểm tra
model.evaluate(X_test, y_test)

# Đánh giá hiệu suất mô hình trên tập kiểm tra
print("Mean Squared Error:", best_model.metrics[1].result().numpy())
print("Root Mean Squared Error:", best_model.metrics[2].result().numpy())
print("Mean Absolute Error:", best_model.metrics[3].result().numpy())
print("R-squared:", best_model.metrics[4].result().numpy())

### Kết luận

sau khi thực hiện train model và ta được kết quả:
- Mean Squared Error: 8465224000.0
- Root Mean Squared Error: 8465224000.0
- Mean Absolute Error: 92006.66
- R-squared: 79019.4


<a name='4'></a>
## 4 - Sử dụng Hyperparameter Tuning để cải tiến model

In [ ]:
!pip install keras-tuner

In [ ]:
from kerastuner.tuners import RandomSearch
# Define build_model function
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate',
                                                             values=[1e-2, 1e-3, 1e-4])),
                  loss='mse',
                  metrics=[MeanSquaredError(), RootMeanSquaredError(), MeanAbsoluteError(), r_squared])
    return model

# Define tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=3,
    directory='kaggle',
    project_name='hoc_ml_mse11')

# Fit tuner to data
tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test))

# Print best hyperparameters
print(tuner.get_best_hyperparameters()[0].values)

In [ ]:
# lấy model tốt nhất
best_model = tuner.get_best_models(num_models=1)[0]

# chạy với tập train
best_model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))


In [ ]:
# chạy thử mô hình với tập kiểm tra
best_model.evaluate(X_test, y_test)

# Đánh giá hiệu suất mô hình trên tập kiểm tra
print("Mean Squared Error:", best_model.metrics[1].result().numpy())
print("Root Mean Squared Error:", best_model.metrics[2].result().numpy())
print("Mean Absolute Error:", best_model.metrics[3].result().numpy())
print("R-squared:", best_model.metrics[4].result().numpy())

### Kết luận

sau khi sử dụng Hyperparameter Turning và ta được kết quả:
- Mean Squared Error: **1471716000.0**   so với lúc đầu 8465224000.0
- Root Mean Squared Error: **1471716000.0**  so với lúc đầu  8465224000.0
- Mean Absolute Error: **38362.95**  so với lúc đầu  92006.66
- R-squared: **26485.656**  so với lúc đầu  79019.4

sai số đã cải tiến dáng kể so với model lúc đầu

<a name='5'></a>
## 5 - Cải tiến Hyperparameter Tuning
- thay đổi giá trị max_trials=10

In [ ]:
# Define tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=3,
    directory='kaggle',
    project_name='hoc_ml_mse11')

# Fit tuner to data
tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test))

# Print best hyperparameters
print(tuner.get_best_hyperparameters()[0].values)

In [ ]:
# lấy model tốt nhất
best_model_2 = tuner.get_best_models(num_models=1)[0]

# chạy với tập train
best_model_2.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

# chạy thử mô hình với tập kiểm tra
best_model_2.evaluate(X_test, y_test)

# Đánh giá hiệu suất mô hình trên tập kiểm tra
print("Mean Squared Error:", best_model.metrics[1].result().numpy())
print("Root Mean Squared Error:", best_model.metrics[2].result().numpy())
print("Mean Absolute Error:", best_model.metrics[3].result().numpy())
print("R-squared:", best_model.metrics[4].result().numpy())

### Kết luận

Sau khi sử dụng thay đổi giá trị max_trials và ta được kết quả:
- Mean Squared Error: **1372885000.0** so với trước đó **1471716000.0**
- Root Mean Squared Error:  **1372885000.0** so với trước đó  **1471716000.0** 
- Mean Absolute Error: **37052.46** so với trước đó **38362.95** 
- R-squared: **24789.904** so với trước đó  **26485.656**
Các chỉ số đã có sự cải thiện hơn so với trước đó.



<a name='5'></a>
## 6 - Cải tiến bằng cách dùng Hyperband

In [ ]:
from kerastuner import Hyperband
# Initialize the Hyperband tuner
tuner = Hyperband(build_model,
                     objective='val_mean_squared_error',
                     max_epochs=10,
                     factor=3,
                     directory='kaggle',
                     project_name='hoc_ml_mse11')

# Perform the hyperparameter search
tuner.search(X_train, y_train, epochs=10, validation_split=0.2)

# Get the best model
best_model_3 = tuner.get_best_models(num_models=1)[0]

In [ ]:
# chạy thử mô hình với tập kiểm tra
best_model_3.evaluate(X_test, y_test)

# Đánh giá hiệu suất mô hình trên tập kiểm tra
print("Mean Squared Error:", best_model.metrics[1].result().numpy())
print("Root Mean Squared Error:", best_model.metrics[2].result().numpy())
print("Mean Absolute Error:", best_model.metrics[3].result().numpy())
print("R-squared:", best_model.metrics[4].result().numpy())

### Kết luận

Sau khi sử dụng Hyperband để cải tiến mô hình:
- Mean Squared Error: 1575010800.0 lớn hơn **1372885000.0**
- Root Mean Squared Error: 1575010800.0 lớn hơn **1372885000.0** 
- Mean Absolute Error: 39686.406 lớn hơn **37052.46**
- R-squared: 27720.771 lớn hơn **24789.904**
Các chỉ số lơn hơn so với mô hình trước đó.
